In [49]:
import numpy as np
import gurobipy as gbp
import time
np.random.seed(352)


In [50]:
n_com=2
n_sup=2
n_dem=3

In [51]:
#Supply Demand of all commodities
Si=np.array([[100,200],[200,700]])
Dj=np.array([[75,190],[150,320],[200,225]])
Si,Dj

(array([[100, 200],
        [200, 700]]), array([[ 75, 190],
        [150, 320],
        [200, 225]]))

In [52]:
#Vehicle capacity, number of vehicles, cost of transportation from supply nodes to demand nodes
V_cap=150
V_num=np.array([5,7])
V_cost=np.array([[10,15,20],[18,13,20]])

In [53]:
#People Variables
n_pep=[25,50,30] #no of people on each demand node,
mu=[4,10] # consumption rate (comm req per day)

In [54]:
# Indices & Variable Names
supply_nodes = n_sup
demand_nodes = n_dem
supply_nodes_range = range(n_sup)
demand_nodes_range = range(n_dem)
comm_range=range(n_com)
all_nodes_len = n_sup*n_dem
ALL_nodes_range = range(all_nodes_len)

print (supply_nodes_range, demand_nodes_range,comm_range, all_nodes_len)

range(0, 2) range(0, 3) range(0, 2) 6


In [55]:
Pc=Dj/Dj.sum(axis=0)
Pc

array([[0.17647059, 0.2585034 ],
       [0.35294118, 0.43537415],
       [0.47058824, 0.30612245]])

In [67]:
# Create Model, Set MIP Focus, Add Variables, & Update Model
m = gbp.Model(' -- The Multi Commodity Vehicle Transportation Problem -- ')

# Set MIP Focus to 2 for optimality
m.setParam('MIPFocus', 2)
# m.setParam(gbp.GRB.Param.PoolSearchMode, 1)
# m.setParam(gbp.GRB.Param.PoolGap, 0.10)

decision_var = []
vehicles_var=[]
unserved_var=[]
for orig in supply_nodes_range:
    decision_var.append([])
    vehicles_var.append([])
    for dest in demand_nodes_range:
        decision_var[orig].append([])
        vehicles_var[orig].append(m.addVar(vtype=gbp.GRB.INTEGER,
                                          name='S'+str(orig+1)+'_D'+str(dest+1)+'_V'))
        for comm in comm_range:
#             print (comm,decision_var)
            decision_var[orig][dest].append(m.addVar(vtype=gbp.GRB.INTEGER, 
#                                         obj=Cij[orig][dest],
#                                            obj=1,
                                        name='S'+str(orig+1)+'_D'+str(dest+1)+'_c'+str(comm+1)))
for dest in demand_nodes_range:
    unserved_var.append([])
    for comm in comm_range:
        unserved_var[dest].append(m.addVar(vtype=gbp.GRB.INTEGER,
                                          name='D'+str(dest+1)+'_c'+str(comm+1)+'_U'))
# Update Model Variables
m.update()       


Changed value of parameter MIPFocus to 2
   Prev: 0  Min: 0  Max: 3  Default: 0


In [68]:
#objective function
m.setObjective(
    gbp.quicksum(gbp.quicksum((int(Dj[dest][comm])-gbp.quicksum(decision_var[orig][dest][comm] for orig in supply_nodes_range))*(Pc[dest][comm])
                            for dest in demand_nodes_range) for comm in comm_range)+
    gbp.quicksum(gbp.quicksum(vehicles_var[orig][dest]*V_cost[orig][dest] for dest in demand_nodes_range) for orig in supply_nodes_range)+
    gbp.quicksum(gbp.quicksum(unserved_var[dest][comm] for comm in comm_range) for dest in demand_nodes_range),
                        gbp.GRB.MINIMIZE)

In [69]:
m.update()

In [70]:
# Add Supply Constraints
for orig in supply_nodes_range:
    for comm in comm_range:
        m.addConstr(gbp.quicksum(decision_var[orig][dest][comm]
                                 for dest in demand_nodes_range) - Si[orig][comm] <= 0)
# Add Demand Constraints
for dest in demand_nodes_range:  
    for comm in comm_range:
        m.addConstr(gbp.quicksum(decision_var[orig][dest][comm] 
                                 for orig in supply_nodes_range) - Dj[dest][comm] <= 0)
#Add vehicle constraints
for orig in supply_nodes_range:
    m.addConstr(gbp.quicksum(decision_var[orig][dest][comm]
                             for dest in demand_nodes_range for comm in comm_range) - V_cap*V_num[orig] <=0)
for orig in supply_nodes_range:
    m.addConstr(gbp.quicksum(vehicles_var[orig][dest] for dest in demand_nodes_range) - V_num[orig] <=0)
    
for orig in supply_nodes_range:
    for dest in demand_nodes_range:
        m.addConstr(-sum(decision_var[orig][dest][comm]
                            for comm in comm_range)/V_cap + vehicles_var[orig][dest]>=0)
for orig in supply_nodes_range:
    for dest in demand_nodes_range:
        m.addConstr(-sum(decision_var[orig][dest][comm]
                            for comm in comm_range)/V_cap + vehicles_var[orig][dest]<=1)
        
#Add unserved people contstraints
for dest in demand_nodes_range:
    for comm in comm_range:
        m.addConstr(unserved_var[dest][comm]<=n_pep[dest])

for dest in demand_nodes_range:
    for comm in comm_range:
        m.addConstr(sum(decision_var[orig][dest][comm] for orig in supply_nodes_range)-
                    ((n_pep[dest]-unserved_var[dest][comm])*mu[comm])>=0)
#  Optimize and Print( Results)
m.optimize()
m.write('path.lp')

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 38 rows, 24 columns and 102 nonzeros
Model fingerprint: 0xa7bb449a
Variable types: 0 continuous, 24 integer (0 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+01]
  Objective range  [2e-01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 15198.537294
Presolve removed 7 rows and 0 columns
Presolve time: 0.01s
Presolved: 31 rows, 24 columns, 90 nonzeros
Variable types: 0 continuous, 24 integer (0 binary)
Presolve removed 15 rows and 6 columns
Presolved: 16 rows, 18 columns, 42 nonzeros


Root relaxation: objective 4.511195e+03, 6 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4511.19487    0    4 15198.5373 4511.19487  70.3%     -    0s
H    0     0                    4541.8348698 4511.19487  0.6

In [46]:
m.display()

Minimize
   <gurobi.LinExpr: 417.60704281712685 + 10.0 S1_D1_V + -0.17647058823529413 S1_D1_c1 + -0.2585034013605442 S1_D1_c2 + 15.0 S1_D2_V + -0.35294117647058826 S1_D2_c1 + -0.43537414965986393 S1_D2_c2 + 20.0 S1_D3_V + -0.47058823529411764 S1_D3_c1 + -0.30612244897959184 S1_D3_c2 + 18.0 S2_D1_V + -0.17647058823529413 S2_D1_c1 + -0.2585034013605442 S2_D1_c2 + 13.0 S2_D2_V + -0.35294117647058826 S2_D2_c1 + -0.43537414965986393 S2_D2_c2 + 20.0 S2_D3_V + -0.47058823529411764 S2_D3_c1 + -0.30612244897959184 S2_D3_c2 + D1_c1_U + D1_c2_U + D2_c1_U + D2_c2_U + D3_c1_U + D3_c2_U>
Subject To
   R0 : <gurobi.LinExpr: S1_D1_c1 + S1_D2_c1 + S1_D3_c1> <= 100.0
   R1 : <gurobi.LinExpr: S1_D1_c2 + S1_D2_c2 + S1_D3_c2> <= 200.0
   R2 : <gurobi.LinExpr: S2_D1_c1 + S2_D2_c1 + S2_D3_c1> <= 200.0
   R3 : <gurobi.LinExpr: S2_D1_c2 + S2_D2_c2 + S2_D3_c2> <= 700.0
   R4 : <gurobi.LinExpr: S1_D1_c1 + S2_D1_c1> <= 75.0
   R5 : <gurobi.LinExpr: S1_D1_c2 + S2_D1_c2> <= 190.0
   R6 : <gurobi.LinExpr: S1_D2_c1 +

In [62]:
type(decision_var[orig][dest][comm].x)

float

In [48]:
selected = {}
Closed = []
for v in m.getVars():
    var = '%s' % v.VarName
    units=int(v.x)
    selected[var] = units
#     print (var,units)
    if (v.Varname[-1]=='V'):
        print ('-'*100)
        print( '|  Supply Facility #', var[:2], 'is sending', units, \
              'vehicles to Demand Facility #', var[3:5]) 
        
    elif v.Varname[-2]=='c':
        print( '|  Supply Facility #', var[:2], 'is shipping', units, \
                                            'units of commodity',var[-2:], 'to Demand Facility #', var[3:5])
    elif v.Varname[-1]=='U':
        print ('-'*100)
        print( '|  Demand Facility #', var[:2], 'is not serving', units, \
                                            'people with commondity#', var[3:5])
    else:
        print ('Hiiiiiiii')
        
print( '******************************************************************************')
print( '    | Objective Value --------------------- ', int(m.objVal))
print( '    | Supply Facilities ------------------- ', len(Si))
print( '    | Total Supply Units ------------------ ', Si.sum())
print( '    | Demand Facilites -------------------- ', len(Dj))
print( '    | Total Demand Units ------------------ ', Dj.sum())
print( '    | Total Potential Combinations -------- ', len(Si)*len(Dj))
print( '    | Actual Combinations  ---------------- ', len(selected))
# print( '    | Real Time to Optimize (sec.) -------- ', t2)
print( '******************************************************************************')
print( '  --  The Transportation Simplex with Gurobi --')


----------------------------------------------------------------------------------------------------
|  Supply Facility # S1 is sending 2 vehicles to Demand Facility # D1
|  Supply Facility # S1 is shipping 0 units of commodity c1 to Demand Facility # D1
|  Supply Facility # S1 is shipping 190 units of commodity c2 to Demand Facility # D1
----------------------------------------------------------------------------------------------------
|  Supply Facility # S1 is sending 0 vehicles to Demand Facility # D2
|  Supply Facility # S1 is shipping 0 units of commodity c1 to Demand Facility # D2
|  Supply Facility # S1 is shipping 0 units of commodity c2 to Demand Facility # D2
----------------------------------------------------------------------------------------------------
|  Supply Facility # S1 is sending 1 vehicles to Demand Facility # D3
|  Supply Facility # S1 is shipping 100 units of commodity c1 to Demand Facility # D3
|  Supply Facility # S1 is shipping 10 units of commodity c2 to